<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Alzheimer-Risk-Assessment-WebApp/blob/master/Defined%20Projects%20With%20Objectives/Project_2_Predictive_Maintenance_For_Industrial/NASA_Turbofan_Maintenance_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Libraries <a id=1></a>

In [137]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

import keras
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense , LSTM, Dropout

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
import math
import xgboost
import time
from tqdm import tqdm

# Setting seed for reproducibility
np.random.seed(1234)
PYTHONHASHSEED = 0

# 2. Download & Import Dataset <a id=2></a>

In [64]:
#!/bin/bash
!kaggle datasets download behrad3d/nasa-cmaps
!unzip nasa-cmaps.zip

Dataset URL: https://www.kaggle.com/datasets/behrad3d/nasa-cmaps
License(s): CC0-1.0
nasa-cmaps.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  nasa-cmaps.zip
replace CMaps/Damage Propagation Modeling.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CMaps/Damage Propagation Modeling.pdf  
  inflating: CMaps/RUL_FD001.txt     
  inflating: CMaps/RUL_FD002.txt     
  inflating: CMaps/RUL_FD003.txt     
  inflating: CMaps/RUL_FD004.txt     
  inflating: CMaps/readme.txt        
  inflating: CMaps/test_FD001.txt    
  inflating: CMaps/test_FD002.txt    
  inflating: CMaps/test_FD003.txt    
  inflating: CMaps/test_FD004.txt    
  inflating: CMaps/train_FD001.txt   
  inflating: CMaps/train_FD002.txt   
  inflating: CMaps/train_FD003.txt   
  inflating: CMaps/train_FD004.txt   
  inflating: CMaps/x.txt             


In [88]:
fd1_train = pd.read_csv("/content/CMaps/train_FD001.txt", sep=' ', header=None)
fd1_test = pd.read_csv("/content/CMaps/test_FD001.txt", sep=" ", header=None)

# 3. Analyzing & Cleaning Dataset <a id=3></a>

In [66]:
fd1_train.describe().T

,count,mean,std,min,25%,50%,75%,max
0,20631.0,51.506568,2.922763e+01,1.0000,26.0000,52.0000,77.0000,100.0000
1,20631.0,108.807862,6.888099e+01,1.0000,52.0000,104.0000,156.0000,362.0000
2,20631.0,-0.000009,2.187313e-03,-0.0087,-0.0015,0.0000,0.0015,0.0087
3,20631.0,0.000002,2.930621e-04,-0.0006,-0.0002,0.0000,0.0003,0.0006
4,20631.0,100.000000,0.000000e+00,100.0000,100.0000,100.0000,100.0000,100.0000
5,20631.0,518.670000,6.537152e-11,518.6700,518.6700,518.6700,518.6700,518.6700
6,20631.0,642.680934,5.000533e-01,641.2100,642.3250,642.6400,643.0000,644.5300
7,20631.0,1590.523119,6.131150e+00,1571.0400,1586.2600,1590.1000,1594.3800,1616.9100
8,20631.0,1408.933782,9.000605e+00,1382.2500,1402.3600,1408.0400,1414.5550,1441.4900
9,20631.0,14.620000,3.394700e-12,14.6200,14.6200,14.6200,14.6200,14.6200


### 3.1 Drop NaNs Columns

In [89]:
fd1_train = fd1_train.drop(columns=[26, 27])
fd1_test = fd1_test.drop(columns=[26, 27])

### 3.2 Define Columns Names

In [90]:
columns = ['unit_number','time_in_cycles','setting_1','setting_2','TRA','T2','T24','T30','T50','P2','P15','P30','Nf',
           'Nc','epr','Ps30','phi','NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32' ]

fd1_train.columns = columns
fd1_test.columns = columns

In [69]:
fd1_train.describe().T

,count,mean,std,min,25%,50%,75%,max
unit_number,20631.0,51.506568,2.922763e+01,1.0000,26.0000,52.0000,77.0000,100.0000
time_in_cycles,20631.0,108.807862,6.888099e+01,1.0000,52.0000,104.0000,156.0000,362.0000
setting_1,20631.0,-0.000009,2.187313e-03,-0.0087,-0.0015,0.0000,0.0015,0.0087
setting_2,20631.0,0.000002,2.930621e-04,-0.0006,-0.0002,0.0000,0.0003,0.0006
TRA,20631.0,100.000000,0.000000e+00,100.0000,100.0000,100.0000,100.0000,100.0000
T2,20631.0,518.670000,6.537152e-11,518.6700,518.6700,518.6700,518.6700,518.6700
T24,20631.0,642.680934,5.000533e-01,641.2100,642.3250,642.6400,643.0000,644.5300
T30,20631.0,1590.523119,6.131150e+00,1571.0400,1586.2600,1590.1000,1594.3800,1616.9100
T50,20631.0,1408.933782,9.000605e+00,1382.2500,1402.3600,1408.0400,1414.5550,1441.4900
P2,20631.0,14.620000,3.394700e-12,14.6200,14.6200,14.6200,14.6200,14.6200


## 3.3 Drop Columns With Constant Values

In [91]:
fd1_train = fd1_train.drop(columns=['Nf_dmd','PCNfR_dmd','P2','T2','TRA','farB','epr'])
fd1_test = fd1_test.drop(columns=['Nf_dmd','PCNfR_dmd','P2','T2','TRA','farB','epr'])

### 3.4 Add `RUL` Varibale

In [92]:
fd_rul = fd1_train.groupby("unit_number")['time_in_cycles'].max().reset_index()
fd_rul.columns = ['unit_number', 'max']
fd1_train = fd1_train.merge(fd_rul, on=['unit_number'], how='left')
fd1_train['RUL'] = fd1_train['max'] - fd1_train['time_in_cycles']
fd1_train.drop(columns=['max'], inplace=True)
fd1_train.head()

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,Nc,Ps30,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187


__Merge With train Set to `df`__

In [93]:
df = fd1_train.copy()
df = df.merge(fd_rul, on=['unit_number'], how='left')
df.head()

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,...,Ps30,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL,max
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,...,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191,192
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,...,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190,192
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,...,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189,192
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,...,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188,192
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,...,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187,192


In [94]:
df['RUL'] = df['max'] - df['time_in_cycles']
df.head()

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,...,Ps30,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL,max
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,...,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191,192
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,...,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190,192
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,...,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189,192
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,...,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188,192
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,...,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187,192


In [124]:
df_train = df.drop(columns = ['unit_number','setting_1','setting_2','P15','NRc'])

### 3.5 Process Test Set

In [131]:
# Preprocess test data
test_df = fd1_test.copy()
test_df.drop(columns=['setting_1', 'setting_2', 'P15', 'NRc'], inplace=True)

# 4. Preparing Data <a id=4></a>

In [96]:
def print_regression_metrics(y_test, y_pred):
    """
    Calculates and prints common regression metrics.

    Parameters:
    - y_test (array-like): The true target values.
    - y_pred (array-like): The predicted target values.

    Returns:
    None
    """
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print("Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

In [97]:
def generate_labels(df, w1=30, w0=15):
    """
    Generates binary labels ('label1' and 'label2') based on Remaining Useful Life (RUL) thresholds.

    Args:
    df (pd.DataFrame): The input DataFrame with the 'RUL' column.
    w1 (int, optional): Threshold for 'label1'. Defaults to 30.
    w0 (int, optional): Threshold for 'label2'. Defaults to 15.

    Returns:
    pd.DataFrame: DataFrame with 'label1' and 'label2' columns added.
    """
    # Generate 'label1' as 1 if RUL <= w1, else 0
    df['label1'] = np.where(df['RUL'] <= w1, 1, 0)

    # Copy 'label1' to 'label2' and set it to 2 where RUL <= w0
    df['label2'] = df['label1']
    df.loc[df['RUL'] <= w0, 'label2'] = 2

    return df

In [125]:
train_df = generate_labels(fd1_train, w1=30, w0=15)
train_df.head()

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,...,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL,label1,label2
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,...,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,...,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,...,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,...,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,...,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187,0,0


In [99]:
def normalize_data(df, columns_to_exclude):
    """
    Normalizes the DataFrame using Min-Max scaling, excluding specified columns.

    Args:
    df (pd.DataFrame): The DataFrame to be normalized.
    columns_to_exclude (list): List of columns to exclude from normalization.

    Returns:
    pd.DataFrame: Normalized DataFrame with the specified columns excluded.
    MinMaxScaler: The scaler used for normalization, to apply to the test set.
    """
    # Get columns to normalize (exclude specified ones)
    cols_normalize = df.columns.difference(columns_to_exclude)

    # Apply MinMax scaling
    min_max_scaler = MinMaxScaler()
    norm_df = pd.DataFrame(min_max_scaler.fit_transform(df[cols_normalize]),
                           columns=cols_normalize,
                           index=df.index)

    # Join the non-normalized columns back with the normalized ones
    return df[columns_to_exclude].join(norm_df).reindex(columns=df.columns), min_max_scaler

In [126]:
cols_to_exclude_train = ['unit_number', 'time_in_cycles', 'RUL', 'label1', 'label2']
train_df, min_max_scaler = normalize_data(train_df, cols_to_exclude_train)
train_df

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,...,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL,label1,label2
0,1,1,0.459770,0.166667,0.183735,0.406802,0.309757,1.0,0.726248,0.242424,...,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662,191,0,0
1,1,2,0.609195,0.250000,0.283133,0.453019,0.352633,1.0,0.628019,0.212121,...,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014,190,0,0
2,1,3,0.252874,0.750000,0.343373,0.369523,0.370527,1.0,0.710145,0.272727,...,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,189,0,0
3,1,4,0.540230,0.500000,0.343373,0.256159,0.331195,1.0,0.740741,0.318182,...,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,188,0,0
4,1,5,0.390805,0.333333,0.349398,0.257467,0.404625,1.0,0.668277,0.242424,...,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,187,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,0.477011,0.250000,0.686747,0.587312,0.782917,1.0,0.254428,0.439394,...,0.170576,0.558824,0.194344,0.656791,0.750000,0.271318,0.109500,4,1,2
20627,100,197,0.408046,0.083333,0.701807,0.729453,0.866475,1.0,0.162641,0.500000,...,0.211087,0.500000,0.188668,0.727203,0.583333,0.124031,0.366197,3,1,2
20628,100,198,0.522989,0.500000,0.665663,0.684979,0.775321,1.0,0.175523,0.515152,...,0.281450,0.529412,0.212148,0.922278,0.833333,0.232558,0.053991,2,1,2
20629,100,199,0.436782,0.750000,0.608434,0.746021,0.747468,1.0,0.133655,0.530303,...,0.208955,0.514706,0.203065,0.823394,0.583333,0.116279,0.234466,1,1,2


In [114]:
# existing_cols_to_exclude = [col for col in cols_to_exclude_train if col in test_df.columns]
# test_df, _ = normalize_data(test_df, existing_cols_to_exclude)
# test_df.head()

In [19]:
def generate_sequences(df, sequence_length, seq_cols):
    """
    Generates sequences of data from a DataFrame for LSTM input.

    Args:
    df (pd.DataFrame): DataFrame containing sensor data.
    sequence_length (int): Length of each sequence.
    seq_cols (list): List of columns to be included in each sequence.

    Yields:
    numpy.ndarray: A sequence of data for each iteration.
    """
    # Convert DataFrame columns to numpy array for sequence generation
    data_matrix = df[seq_cols].values
    num_elements = data_matrix.shape[0]

    # Generate sequences of fixed length
    for start, stop in zip(range(0, num_elements - sequence_length), range(sequence_length, num_elements)):
        yield data_matrix[start:stop, :]

In [127]:
sequence_length = 50
sequence_cols = list(train_df.columns.difference(['unit_number', 'time_in_cycles', 'RUL', 'label1', 'label2']))

# Generate sequences for training data
seq_array = np.concatenate([
    list(generate_sequences(train_df[train_df['unit_number'] == unit], sequence_length, sequence_cols))
    for unit in train_df['unit_number'].unique()
]).astype(np.float32)

In [120]:
def generate_label_array(df, sequence_length, label_col):
    """
    Generate label arrays for sequence-based RUL prediction.
    Arguments:
    - df: A DataFrame for a single unit.
    - sequence_length: The length of the sequences for prediction.
    - label_col: The column name for the label (e.g., 'RUL').
    Returns:
    - A NumPy array of labels corresponding to the sequences.
    """
    data_matrix = df[label_col].values  # Extract label values as a NumPy array
    num_elements = data_matrix.shape[0]  # Total number of elements in the unit

    # Ensure we have enough data for at least one sequence
    if num_elements <= sequence_length:
        return np.array([])  # No labels can be created for this unit

    # Return labels starting from the point after the sequence_length
    return data_matrix[sequence_length:num_elements]  # Slice the 1D array correctly

In [128]:
# Generate label array for training data
label_array = np.concatenate([
    generate_label_array(train_df[train_df['unit_number'] == unit], sequence_length, 'RUL')
    for unit in train_df['unit_number'].unique()
]).astype(np.float32)

In [103]:
def calculate_rul_for_test_data(test_df, truth_df):
    """
    Calculates the Remaining Useful Life (RUL) for the test data by merging with ground truth.

    Args:
    test_df (pd.DataFrame): The test dataset with sensor readings.
    truth_df (pd.DataFrame): The ground truth dataset with RUL values.

    Returns:
    pd.DataFrame: The test DataFrame with the 'RUL' column added.
    """
    # Calculate the max time for each unit in the test data
    rul = pd.DataFrame(test_df.groupby('unit_number')['time_in_cycles'].max()).reset_index()
    rul.columns = ['unit_number', 'max']

    # Merge with the ground truth DataFrame
    truth_df['unit_number'] = truth_df.index + 1
    truth_df['max'] = rul['max'] + truth_df['more']
    truth_df.drop('more', axis=1, inplace=True)

    # Merge RUL values into the test data
    test_df = test_df.merge(truth_df, on=['unit_number'], how='left')
    test_df['RUL'] = test_df['max'] - test_df['time_in_cycles']
    test_df.drop('max', axis=1, inplace=True)

    return test_df

In [134]:
def lstm_data_preprocessing(raw_train_data, raw_test_data, raw_RUL_data, sequence_length=50, w1=30, w0=15):
    """
    Preprocesses the training and test data for LSTM input, including normalization, sequence generation, and label creation.

    Args:
    raw_train_data (pd.DataFrame): The raw training dataset.
    raw_test_data (pd.DataFrame): The raw test dataset.
    raw_RUL_data (pd.DataFrame): The raw RUL dataset (truth values).
    sequence_length (int, optional): The sequence length for LSTM. Defaults to 50.
    w1 (int, optional): Threshold for 'label1'. Defaults to 30.
    w0 (int, optional): Threshold for 'label2'. Defaults to 15.

    Returns:
    tuple: A tuple containing:
        - seq_array (numpy.ndarray): The input sequences for LSTM.
        - label_array (numpy.ndarray): The label array for LSTM.
        - test_df (pd.DataFrame): The processed test data with RUL.
        - sequence_length (int): The sequence length used.
        - sequence_cols (list): The columns used for generating sequences.
    """
    # Preprocess Training Data
    train_df = raw_train_data
    truth_df = raw_RUL_data

    # Safely drop extra columns if present
    if truth_df.shape[1] > 1:
        truth_df.drop(truth_df.columns[1], axis=1, inplace=True)

    # Generate labels for train data
    train_df = generate_labels(train_df, w1, w0)

    # Normalize training data
    cols_to_exclude_train = ['unit_number', 'time_in_cycles', 'RUL', 'label1', 'label2']
    train_df, min_max_scaler = normalize_data(train_df, cols_to_exclude_train)
    print("Train Data Preprocessing Complete")

    # Preprocess Test Data
    test_df = raw_test_data.drop(columns=['setting_1', 'setting_2', 'P15', 'NRc'], errors='ignore')
    test_df, _ = normalize_data(test_df, cols_to_exclude_train)  # Normalizing test data with the same scaler
    print("Test Data Preprocessing Complete")

    # Calculate RUL for test data
    test_df = calculate_rul_for_test_data(test_df, truth_df)

    # Generate labels for test data
    test_df = generate_labels(test_df, w1, w0)

    # Generate sequences for LSTM input (train data)
    sequence_cols = list(test_df.columns[:-3])
    seq_array = np.concatenate([list(generate_sequences(train_df[train_df['unit_number'] == id], sequence_length, sequence_cols))
                               for id in train_df['unit_number'].unique()]).astype(np.float32)
    print("Sequences Generated")

    # Generate label array for train data
    label_array = np.concatenate([generate_label_array(train_df[train_df['unit_number'] == id], sequence_length, ['RUL'])
                                  for id in train_df['unit_number'].unique()]).astype(np.float32)

    return seq_array, label_array, test_df, sequence_length, sequence_cols


**Processing Test Data**

In [129]:
# Ensure fd_rul has correct columns
fd_rul = fd1_test.groupby('unit_number')['time_in_cycles'].max().reset_index()
fd_rul.columns = ['unit_number', 'more']  # Rename for compatibility

In [113]:
# Preprocess test data
test_df = fd1_test.copy()
test_df.drop(columns=['setting_1', 'setting_2', 'P15', 'NRc'], inplace=True)

# Calculate RUL for test data
fd_rul = test_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
fd_rul.columns = ['unit_number', 'more']  # Rename for compatibility
test_df = calculate_rul_for_test_data(test_df, fd_rul)

# Generate labels for test data
test_df = generate_labels(test_df, w1=30, w0=15)

# Dynamically find columns to exclude for normalization
cols_to_exclude = [col for col in ['unit_number', 'time_in_cycles', 'RUL', 'label1', 'label2'] if col in test_df.columns]

# Normalize
test_df, _ = normalize_data(test_df, cols_to_exclude)

**Combine Within One Function**

In [136]:
seq_array, label_array, test_df, sequence_length, sequence_cols = lstm_data_preprocessing(
    raw_train_data=df_train,
    raw_test_data=test_df,
    raw_RUL_data=fd_rul,
    sequence_length=50,
    w1=30,
    w0=15
)


KeyError: "['unit_number'] not in index"